In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Project_DevC
!ls

/content/drive/My Drive/Project_DevC
clone_cmt_tiki	clone_data  scrapingweb_bookingcomment.ipynb


In [3]:
#import library
from bs4 import BeautifulSoup
import requests
import json
import csv
import pandas as pd
import urllib, json

In [22]:
names = ['ao+khoac+mua+dong', 'bàn+là', 'tủ+quần+áo', 'sach+văn+học', 'chảo+chống+dính', 'dầu+rửa+bát', 'xà+phòng',
         'máy+vi+tính', 'tủ+lạnh', 'quạt+trần', 'hambuger', 'bánh+nướng+trung+thu', 'tinh+dầu+tràm', 'khóa+cửa', 'xe+máy', 'nước+giải+khát', "gà+rán", "bánh+đa"
         , 'áo+vest', 'nước+xả+vải', 'kéo', 'bánh+cuốn', 'dầu+gội', 'máy+lọc+nước']

In [27]:
names = ['bánh+nướng+trung+thu', 'tinh+dầu+tràm', 'khóa+cửa', 'xe+máy', 'nước+giải+khát', "gà+rán", "bánh+đa"
         , 'áo+vest', 'nước+xả+vải', 'kéo', 'bánh+cuốn', 'dầu+gội', 'máy+lọc+nước']

In [28]:
def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        print("Crawl page: {} {}".format(i, name))
        response = requests.get(laptop_page_url.format(i))
        parser = BeautifulSoup(response.text, 'html.parser')

        product_box = parser.findAll(class_="product-item")

        if (len(product_box) == 0):
            break

        for product in product_box:
            product_list.append(product.get("data-id"))

        i += 1
        if (i == 2):
          break


    return product_list, i

def save_product_id(product_list=[]):
    file = open(product_id_file, "w+")
    str = "\n".join(product_list)
    file.write(str)
    file.close()
    print("Save file: ", product_id_file)

def crawl_product(product_list=[]):
    product_detail_list = []
    for product_id in product_list:
        response = requests.get(product_url.format(product_id))
        if (response.status_code == 200):
            product_detail_list.append(response.text)
            print("Crawl product: ", product_id, ": ", response.status_code)
    return product_detail_list



In [29]:
for name in names:
  for i in range(1,4):
    laptop_page_url = "https://tiki.vn/search?q=" + name + '&order=top_seller&page=' + str(i)
    product_url = "https://tiki.vn/search?q=" + name
    product_id_file = "/content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/" + name + str(i) + "_id.txt"
    product_list, page = crawl_product_id()
    save_product_id(product_list)

Crawl page: 1 bánh+nướng+trung+thu
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/bánh+nướng+trung+thu1_id.txt
Crawl page: 1 bánh+nướng+trung+thu
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/bánh+nướng+trung+thu2_id.txt
Crawl page: 1 bánh+nướng+trung+thu
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/bánh+nướng+trung+thu3_id.txt
Crawl page: 1 tinh+dầu+tràm
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/tinh+dầu+tràm1_id.txt
Crawl page: 1 tinh+dầu+tràm
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/tinh+dầu+tràm2_id.txt
Crawl page: 1 tinh+dầu+tràm
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/tinh+dầu+tràm3_id.txt
Crawl page: 1 khóa+cửa
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/khóa+cửa1_id.txt
Crawl page: 1 khóa+cửa
Save file:  /content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_pro

In [30]:
list_data = []
def get_dict_from_id_list(df):
  for id in df[0]:
    url = 'https://tiki.vn/api/v2/reviews?product_id=' + str(id) + "&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&limit=10&include=comments"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    list_data.append(data)
  return list_data

In [20]:
#names = ['giay+cao+got', 'sach+giao+khoa', 'truyện+tranh', 'ao+somi', 'do+dung+hoc+tap',
 #        'mu+bao+hiem', 'hoodie', 'ao+len', 'tat', 'tai+nghe+bluetooth', 'tinh+dầu', 'balo', 'quan+au', 'noi+com+dien', "kho+ga", "laptop"
  #       , 'tã+quần', 'bột+giặt', 'bàn+học', 'đèn+học', 'lăn+khử+mùi', 'nước+hoa']

In [15]:
incompleteread = ['giay+nam']

In [31]:
for name in names:
  for i in range(1,4):
    contents = []
    titles = []
    ratings = []
    product_id_file = "/content/drive/My Drive/Project_DevC/clone_cmt_tiki/id_product/" + name + str(i) + "_id.txt"
    df = pd.read_csv(product_id_file, header=None)
    list_data_loop = get_dict_from_id_list(df)
    for data in list_data_loop:
      for dict_ in data['data']:
        for key, value in dict_.items():
          if key == 'content':
            contents.append(value)
          if key == 'rating':
            ratings.append(value)
          if key == 'title':
            titles.append(value)
    data_save = pd.DataFrame({'ratings': ratings, 'titles': titles, 'comments': contents})
    data_frame_notna = data_save.dropna()
    path = "/content/drive/My Drive/Project_DevC/clone_cmt_tiki/data_csv/" + name + str(i) + ".csv"
    data_frame_notna.to_csv(path, index=False)
    print("Saved ", name)


Saved  bánh+nướng+trung+thu
Saved  bánh+nướng+trung+thu
Saved  bánh+nướng+trung+thu
Saved  tinh+dầu+tràm
Saved  tinh+dầu+tràm
Saved  tinh+dầu+tràm
Saved  khóa+cửa
Saved  khóa+cửa
Saved  khóa+cửa
Saved  xe+máy
Saved  xe+máy
Saved  xe+máy
Saved  nước+giải+khát
Saved  nước+giải+khát
Saved  nước+giải+khát
Saved  gà+rán


EmptyDataError: ignored

In [ ]:
#save data as csv file
path = '/content/drive/My Drive/Project_DevC/clone_data/data_comment_tiki/' + name + '1' + '.csv'
data_frame_notna.to_csv(path, index=False)